In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-a8g_k3kn because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
from utils import *
from model import BoltDetector
from dataset import *
from data_module import *

In [3]:
df = pd.read_csv('dataset_strat.csv')

In [4]:
df["bboxes"] = df["bboxes"].apply(eval) # DONT forget this cell

In [5]:
model = BoltDetector(df,num_classes=1,learning_rate=0.001, img_size=1024)


{'name': 'tf_efficientnetv2_l', 'backbone_name': 'tf_efficientnetv2_l', 'backbone_args': {'drop_path_rate': 0.2}, 'backbone_indices': None, 'image_size': [1024, 1024], 'num_classes': 1, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': 'same', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 3, 'fpn_cell_repeats': 3, 'fpn_channels': 88, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': True, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, 'box_loss_weight': 50.0, 'soft_nms': False, 'max_detection_points': 5000, 'max_det_per_image': 100, 'url': '

In [6]:
#save_path = '/home/astamoulakatos/detector/'

In [7]:
#path = '/home/astamoulakatos/detector/detector-epoch=97-valid_loss=0.16.ckpt'

In [8]:
#model = model.load_from_checkpoint(path)

In [9]:
#PyTorch model check point callback function
#Referencehttps://pytorch-lightning.readthedocs.io/en/latest/_modules/pytorch_lightning/trainer/trainer.html

checkpoint_callback = ModelCheckpoint(
    monitor='valid_loss',
    dirpath= save_path,
    filename= '{epoch}-{valid_loss:.2f}',
    save_top_k=2,
    mode='min',
)

In [10]:
tb_logger = TensorBoardLogger('bolt_logs/', name = 'BoltDetector')
lr_monitor = LearningRateMonitor(logging_interval='step')

#PyTorch lightning trainer
#Reference https://pytorch-lightning.readthedocs.io/en/latest/_modules/pytorch_lightning/trainer/trainer.html

trainer = pl.Trainer(
    max_epochs = 10,
    #gpus = 2,
    accelerator="cpu",
    #devices=10,
    #strategy="ddp_spawn",
    #strategy="dp",
    logger = tb_logger,
    callbacks=[lr_monitor, checkpoint_callback]
)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1814: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=2)`.
  rank_zero_warn(


In [11]:
from multiprocessing.managers import SharedMemoryManager
smm = SharedMemoryManager()
smm.start()
sl = smm.ShareableList(range(4))
sl

ShareableList([0, 1, 2, 3], name='psm_365fdd5c')

In [12]:
raw_shm = smm.SharedMemory(size=128000000)

In [13]:
trainer.fit(model)


  | Name  | Type          | Params
----------------------------------------
0 | model | DetBenchTrain | 116 M 
----------------------------------------
116 M     Trainable params
0         Non-trainable params
116 M     Total params
467.619   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/torch/cuda/__init__.py:145: UserWarning: 
NVIDIA GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
/opt/conda/lib/python3.9/site-packages/effdet/bench.py:55: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  indices_all = cls_topk_indices_all // num_clas

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]